<h2>General Definitions</h2>

In [50]:
#Packages
import numpy as np
#Static variables
#Maximum population size
MAX_POPULATION = 1280
#Number of runs for reliable results
NUM_RUNS = 20
#Threshold for number of runs out of NUM_RUNS that need to be successfull
THRESH_NUM = 19
#Uniform crossover probability of inheriting from parent 1, where 1 denotes 100$
UNICROSS = 0.5
#Deceptive Trap Function Mapping
DECEPTIVE = {
    4:4,3:0,2:1,1:2,0:3
}
#Non-Deceptive Trap Function Dict
NON_DECEPTIVE = {
    4:4,3:0,2:0.5,1:1,0:1.5
}
#Block size for trap functions
BLOCK_SIZE = 4

In [32]:
#Crossover Functions

'''
Input: parent1, parent2 two strings of equal length
iterate bitwise over parent string to create child1 and child2. 
For child1 for each bit choose randomly whether to inherit from parent1 or parent2, make opposite choice for child2
'''
def uniformCrossover(parent1,parent2):
    assert(len(parent1)==len(parent2))
    r,c1,c2 = np.random.random_sample((len(parent1),)),"",""
    for i in range(0,len(parent1)):
        if(r[i]>UNICROSS):
            c1+=(parent1[i])
            c2+=(parent2[i])
        else:
            c1+=(parent2[i])
            c2+=(parent1[i])        
    return(c1,c2)

'''
Input: parent1, parent2 two strings of equal length
Randomly choose two indexes, child1 receives information between theese indices from parent2, information outside of theese indices from parent1. Flip for child2

'''
def twoPointCross(parent1,parent2):
    assert(len(parent1)==len(parent2))
    i1 = np.random.randint(len(parent1))
    i2 = np.random.randint(i1,len(parent1))
    print(i1,i2)
    c1_1=parent1[0:i1]
    c2_1=parent2[0:i1]
    c1_2=parent2[i1:i2]
    c2_2=parent1[i1:i2]
    c1_3=parent1[i2::]
    c2_3=parent2[i2::]
    c1=c1_1+c1_2+c1_3
    c2=c2_1+c2_2+c2_3
    return c1,c2
        
#Some basic tests
print(uniformCrossover("AAAAAAAAAAAA","BBBBBBBBBBBB"))
print(uniformCrossover("AAAAAAAGGGGGG","BBBBBBBCCCCCC"))
#print(uniformCrossover("AAAAAAAGGGG","BBBBBBOOOOOO")) # -correctly throws error due to missmatched link
print(twoPointCross("AAAAAAAAAAAA","BBBBBBBBBBBB"))
print(twoPointCross("AAAAAAAGGGGGG","BBBBBBBCCCCCC"))
#print(twoPointCross("AAAAAAAGGGG","BBBBBBOOOOOO")) #-correctly throws error due to missmatched link


In [71]:
#Fitness Functions

def counting_ones(sequ):
    res = 0
    for e in sequ:
        if(e=="1"):
            res+=1
    return res

'''
Given a sequence sequ, and a mapping (NON_DECEPTIVE or DECEPTIVE) calculate the fittness of the sequence
'''
def tightlyLinkedTrapFunction(sequ,mapping):
    res = 0
    for i in range(0,int(len(sequ)/BLOCK_SIZE)):
        si = i*BLOCK_SIZE
        ei = si+BLOCK_SIZE
        block = sequ[si:ei]
        block_fittness = mapping[counting_ones(block)]
        #print(block,block_fittness,res)
        res+=block_fittness
    return res

def looslyLinkedTrapFunction(sequ,mapping):
    n_blocks = int(len(sequ)/BLOCK_SIZE)
    res = 0
    for i in range(0,n_blocks):
        block = ""
        for j in range(0,BLOCK_SIZE):
            #print(i+n_blocks*j)
            block+=sequ[i+n_blocks*j]
        #print(block)
        block_fittness = mapping[counting_ones(block)]
        res+=block_fittness
    return res
        

#Some tests
assert(counting_ones("1111000011110000")==8)
assert(counting_ones("0000000000000000")==0)
assert(counting_ones("0000000000000111")==3)
assert(counting_ones("1111111111111111")==16)

assert(tightlyLinkedTrapFunction("1111000011110000",DECEPTIVE)==14)
assert(tightlyLinkedTrapFunction("1111000011110000",NON_DECEPTIVE)==11)
assert(tightlyLinkedTrapFunction("0000000000000000",DECEPTIVE)==12)
assert(tightlyLinkedTrapFunction("0000000000000000",NON_DECEPTIVE)==6)
assert(tightlyLinkedTrapFunction("0000011100000111",DECEPTIVE)==6)
assert(tightlyLinkedTrapFunction("0000011100000111",NON_DECEPTIVE)==3)
assert(tightlyLinkedTrapFunction("00000001001101111111",DECEPTIVE)==10)
assert(tightlyLinkedTrapFunction("00000001001101111111",NON_DECEPTIVE)==7)

assert(looslyLinkedTrapFunction("1111000011110000",DECEPTIVE)==4)#
assert(looslyLinkedTrapFunction("1111000011110000",NON_DECEPTIVE)==2)
assert(looslyLinkedTrapFunction("0000000000000000",DECEPTIVE)==12)
assert(looslyLinkedTrapFunction("0000000000000000",NON_DECEPTIVE)==6)
assert(looslyLinkedTrapFunction("0111001100010000",DECEPTIVE)==6)
assert(looslyLinkedTrapFunction("0111001100010000",NON_DECEPTIVE)==3)
assert(looslyLinkedTrapFunction("1111011100110001",DECEPTIVE)==7)
assert(looslyLinkedTrapFunction("1111011100110001",NON_DECEPTIVE)==5.5)
assert(looslyLinkedTrapFunction("11111011110011100011",DECEPTIVE)==11) #1,2,3,4,4
assert(looslyLinkedTrapFunction("11111011110011100011",NON_DECEPTIVE)==9.5) #1,2,3,4,4
assert(looslyLinkedTrapFunction("01111001110001100001",DECEPTIVE)==10)#0,1,2,3,4
assert(looslyLinkedTrapFunction("01111001110001100001",NON_DECEPTIVE)==7)#0,1,2,3,4